# Definición, tipologías y casos de uso de Graph Neural Networks para el aprendizaje basado en relaciones: presentación de implementación

## Introducción al notebook

Este documento de tipo notebook acompaña a la memoria del TFG con título *Definición, tipologías y casos de uso de Graph Neural Networks para el aprendizaje basado en relaciones*. El objetivo del documento es presentar la implementación de las pruebas realizadas con diferentes modelos de redes neuronales gráficas explicadas en el proyecto, así como plantear distintas aproximaciones para su codificación.

Aunque van a explicarse algunos conceptos relacionados con las librerías de Python utilizadas, se deja en manos de la memoria de proyecto el explicar la base teórica de los mismos, así como la evaluación e interpretación de los resultados obtenidos.

## Estructura del documento

Mediante el uso del orden presentado en la memoria, se van a repasar distintas aproximaciones a las GNN con dos planteamientos principales:

1. Clasificación de nodos dentro de un grafo:
    1. Implementación mediante un modelo con base espectral.
    1. Implementación con un modelo espacial mediante paso de mensajes.
    1. Contraste de resultados frente a un modelo tradicional que no tiene en cuenta la estructura de los datos, en busca de justificar la motivación de la existencia de las redes gráficas.
1. Predicción de enlazado.
    1. Implementación de un modelo que hace uso de paso de mensajes para predecir enlaces entre nodos.

Para cada uno de estos ejemplos se presentarán resultados de métricas de precisión durante el proceso de entrenamiento, para lo que se hará uso de datos de validación, así como pruebas finales sobre datos de test.

## Preparación del entorno

### Instalación de módulos

Como primer paso, se procede a instalar los módulos necesarios:

* [numpy](https://numpy.org/), para poder realizar las operaciones necesarias sobre los datos.
* [TensorFlow](https://www.tensorflow.org/), como librería principal para ejecutar los algoritmos de aprendizaje computacional del bloque de clasificación de nodos.
* [Spektral](https://graphneural.network/), una cómoda librería y colección de sets de datos que facilita el trabajo con redes neuronales gráficas.
* [PyTorch](https://pytorch.org/), librería de aprendizaje computacional que será usada para las pruebas de predicción de enlaces. Junto a ella se instalarán también algunas sublibrerías que serán necesarias.
* [tqdm](https://github.com/tqdm/tqdm), una librería simple para crear barras de progreso, con el simple objetivo de facilitar la lectura de algunas pruebas.

A continuación, se hacen las llamadas oportunas para instalar las mismas en el entorno actual. De manera adicional, se adjunta el fichero `requirements.txt` junto con este notebook para su uso en caso de necesidad.

In [1]:
# Install required modules
necessary_modules = ['numpy', 'tensorflow', 'spektral', 'torch', 'torch-cluster', 'torch-scatter', 'torch-sparse', 'torch-geometric', 'tqdm']
for module in necessary_modules:
    !pip install {module}

# Import numpy module, that is going to be use across the project
import numpy as np

### Configuración de TensorBoard

Tanto en los ejemplos donde se hace uso de TensorFlow como en aquellos implementados con PyTorch, se extraerán las métricas de aprendizaje y precisión a ficheros de registro que podrán ser leídos mediante [TensorBoard](https://www.tensorflow.org/tensorboard?hl=en), un entorno del propio TensorFlow que facilita su seguimiento y visualización. 

Para ello, es preciso preparar un directorio donde guardar el historial de registro de mensajes y configurar el propio entorno.

In [2]:
# Import TensorBoard callback to be able to use it in all the code samples
from keras.callbacks import TensorBoard

# Prepare placement for the logs
import os
root_logdir = os.path.join(os.curdir, 'my_logs')

# Create directory if it doesn't exists
from pathlib import Path
Path(root_logdir).mkdir(parents=True, exist_ok=True)

# Define a function that returns the path to store the log depending on the experiment
def get_run_logdir(experiment_name):
    import time
    run_id = time.strftime(f'run_{experiment_name}_%Y_%m_%d-%H_%M_%S')
    return os.path.join(root_logdir, run_id) 

## Implementación de pruebas

A continuación se presentan las distintas pruebas realizadas.

### Pruebas de clasificación de nodos

Tal como se expone en la memoria de proyecto, las redes neuronales gráficas permiten, entre otras cosas, la clasificación de nodos en base a la información de sus vecindarios, lo que permite asignar clases dentro del contexto de las entidades de un grafo.

Para la realización de estas pruebas se hará uso de la librería Spektral, que ya ha sido instalada con anterioridad. Spektral ofrece una amplia colección de modelos de red neuronal gráfica ya implementados, así como distintas utilidades de código para su explotación. A modo de añadido, también incorpora una clase especial para poder cargar juegos de datos con distintas características, entre los que se encuentra CORA, que ha sido escogido para la realización de estas pruebas.

Así, el primer paso será importar el juego de datos y repasarlo, para lo que será preciso cargar la propia librería Spektral, así como TensorFlow y Keras, que pasan a sumarse a numpy, ya cargado, a la lista de módulos utilizados.

In [3]:
# Import necessary librearies and define aliases to refer to them easier
import spektral
import tensorflow as tf
from tensorflow import keras

#### Carga y preparación del conjunto de datos

Como se indica en la memoria de proyecto, se hará uso del juego de datos conocido como CORA. Gracias a Spektral, es sencillo descargar este juego de datos y sus distintas estructuras ya preparadas.

In [4]:
# Download CORA dataset and its different members
dataset = spektral.datasets.citation.Citation(
    'cora', 
    random_split=False, # split randomly: 20 nodes per class for training, 30 nodes 
        # per class for validation; or "Planetoid" (Yang et al. 2016)
    normalize_x=False,  # normalize the features
    dtype=np.float32 # numpy data type for the graph data
    )

# Also load a list of labels as names, justo to be able to use it
label_names = ['Case_Based', 'Genetic_Algorithms', 'Neural_Networks', 
    'Probabilistic_Methods', 'Reinforcement_Learning', 'Rule_Learning', 'Theory']

/Users/ghostmou/virtualenvs/uoc-tfg-gnn/lib/python3.9/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Tras esto, se pueden consultar las características del conjunto de datos que ha sido descargado.

In [5]:
print(f'Dataset: {dataset}')
print('First graph: {}'.format(dataset.graphs[0]))
print('Node labels: {}'.format(', '.join(label_names)))
print(f'Training samples: {np.sum(dataset.mask_tr)}')
print(f'Validation samples: {np.sum(dataset.mask_va)}')
print(f'Test samples: {np.sum(dataset.mask_te)}')

Dataset: Citation(n_graphs=1)
First graph: Graph(n_nodes=2708, n_node_features=1433, n_edge_features=None, n_labels=7)
Node labels: Case_Based, Genetic_Algorithms, Neural_Networks, Probabilistic_Methods, Reinforcement_Learning, Rule_Learning, Theory
Training samples: 140
Validation samples: 500
Test samples: 1000


Gracias a los datos extraídos, se puede ver que la variable `dataset` contiene un grafo, el cuál puede ser accedido para conocer sus características. De este modo, se comprueba que el conjunto de datos CORA ha sido descargado y está compuesto por:

* 2708 nodos o vértices, que conforman el grafo.
* 1433 atributos de nodo.
* 0 atributos de relación, es decir, los enlaces que relacionan los nodos no contienen información. Como se explica en la memoria de proyecto, esto impide tener distintas categorías en los vínculos y limitará el aprendizaje al contenido de los nodos y el contexto con el que se relacionan.
* 7 clases. En base a la definición del juego de datos, se sabe que los vértices se clasifican en dicho número de grupos. Sin embargo, como se explica en la memoria de proyecto, también es posible hacer uso de GNNs para clasificar grafos, por lo que Spektral nos permite trabajar con dos tipos de etiquetas: de nodo y de grafo.

Además, Spektral se encarga de generar los subconjuntos del juego de datos que son necesarios para las pruebas a realizar, que quedan divididos en tres:

* Conjunto de entrenamiento, que cuenta con 140 nodos. Es accesible desde `dataset.mask_tr`.
* Conjunto de validación, compuesto por 500 vértices. Puede recuperarse mediante `dataset.mask_va`.
* Conjunto de pruebas o *test*, formado por 1000 entidades. Se pueden computar desde `dataset.mask_te`.

Esta separación en conjuntos de entrenamiento, validación y test se hace mediante un enmascaramiento: están formados por un vector de tipo `narray` con tantas posiciones como nodos hay en el conjunto de datos. En cada posición se registrará un `1` si el vértice correspondiente pertenece al conjunto y con un `0` en caso contrario.

Por esta razón, se presenta la posibilidad de generar los pesos de cada vértice según si han sido seleccionados o no. Con el fin de fijar el mismo peso para todos dentro de cada conjunto, se procede a dividir cada uno de ellos por el número de observaciones en su grupo. El resultado es almacenado en `weighted_mask`.

In [6]:
weighted_mask = [
    mask.astype(np.float32) / np.count_nonzero(mask)
    for mask in (dataset.mask_tr, dataset.mask_va, dataset.mask_te)
]

print(f'Training samples weights: {np.nanmean(np.where(weighted_mask[0] > 0, weighted_mask[0],np.nan), 0)}')
print(f'Validation samples weights: {np.nanmean(np.where(weighted_mask[1] > 0, weighted_mask[1],np.nan), 0)}')
print(f'Test samples weights: {np.nanmean(np.where(weighted_mask[2] > 0, weighted_mask[2],np.nan), 0)}')

Training samples weights: 0.0071428571827709675
Validation samples weights: 0.0020000000949949026
Test samples weights: 0.0010000001639127731


Dentro del grafo registrado, es posible acceder a las diferentes estructuras que lo representan

In [7]:
print('Características de los nodos, en formato matriz:\n{}\n\n'.format(dataset.graphs[0].x))
print('Matriz de adyacencia, de tipo sparse matrix:\n{}\n\n'.format(dataset.graphs[0].a))

Características de los nodos, en formato matriz:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


Matriz de adyacencia, de tipo sparse matrix:
  (0, 633)	1.0
  (0, 1862)	1.0
  (0, 2582)	1.0
  (1, 2)	1.0
  (1, 652)	1.0
  (1, 654)	1.0
  (2, 1)	1.0
  (2, 332)	1.0
  (2, 1454)	1.0
  (2, 1666)	1.0
  (2, 1986)	1.0
  (3, 2544)	1.0
  (4, 1016)	1.0
  (4, 1256)	1.0
  (4, 1761)	1.0
  (4, 2175)	1.0
  (4, 2176)	1.0
  (5, 1629)	1.0
  (5, 1659)	1.0
  (5, 2546)	1.0
  (6, 373)	1.0
  (6, 1042)	1.0
  (6, 1416)	1.0
  (6, 1602)	1.0
  (7, 208)	1.0
  :	:
  (2694, 431)	1.0
  (2694, 2695)	1.0
  (2695, 431)	1.0
  (2695, 2694)	1.0
  (2696, 2615)	1.0
  (2697, 986)	1.0
  (2698, 1400)	1.0
  (2698, 1573)	1.0
  (2699, 2630)	1.0
  (2700, 1151)	1.0
  (2701, 44)	1.0
  (2701, 2624)	1.0
  (2702, 186)	1.0
  (2702, 1536)	1.0
  (2703, 1298)	1.0
  (2704, 641)	1.0
  (2705, 287)	1.0
  (2706, 165)	1.0
  (2706, 169)	1.0
  (

In [8]:
print('Características de las aristas, en formato matriz (vacía, en este caso):\n{}\n\n'.format(dataset.graphs[0].e))
print('Etiquetas de cada nodo:\n{}'.format(dataset.graphs[0].y))

Características de las aristas, en formato matriz (vacía, en este caso):
None


Etiquetas de cada nodo:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


#### Experimento 1: Implementación ConvGNN espectral

Gracias al fichero de datos preparado, es posible probar la implementación de uno de los modelos presentados en la memoria de proyecto: una red neuronal convolucional gráfica o ConvGNN con una aproximación espectral.

Como se explica en el documento de proyecto, el uso de la estructural espectral permite el aprendizaje y predicción en base a la estructura general del grafo, por lo que se hará uso de la matriz de adyacencia y las características de los nodos, pero sin la posibilidad de tener en cuenta la estructura general, algo que veremos en mayor detalle en la implementación de una ConvGNN espacial en el siguiente apartado.

El primer paso será cargar los datos en un `Loader`, una clase dentro de Spektral que se encarga de gestionar el conjunto de datos y devolver los subconjuntos necesarios para el entrenamiento de la red neuronal. Así, se procede a crear uno para el conjunto de entrenamiento y otro para el de validación. Puesto que el ejemplo está destinado a aprender de una única representación de grafo, se usará un `SingleLoader` y se le pasarán los pesos de los nodos, con el fin de tenerlos en cuenta en la salida.

In [9]:
from spektral.data.loaders import SingleLoader
loader_training = SingleLoader(dataset, sample_weights=weighted_mask[0])
loader_validation = SingleLoader(dataset, sample_weights=weighted_mask[1])

Durante la fase de entrenamiento, cada loader devolverá una tupla compuesta por una tupla que contendrá los inputs del modelo, las etiquetas de cada nodo y los pesos de los mismos desde la estructura pasada como `sample_weights`.

Con los datos listos para ser usados, el siguiente paso será crear el modelo desde Spektral. En este caso, se hará uso de la clase `GCN` de Spektral, la cuál hace uso de la arquitectura propuesta en el paper de Kipf y Welling de 2016 (**REFERENCIA** desde memoria). La instancia se crea mediante el paso de:

1. Número de etiquetas de nodo mediante `n_labels`, dato que se recoge desde el conjunto de datos creado.
1. Número de características de nodos mediante `n_input_channels`. Tal como se explica en la memoria, este modelo básico de red convolucional gráfica hace uso de las características de cada nodo y las relaciones de su entorno, pero no tiene en cuenta las propiedades de los enlaces entre vértices, por lo que estas son ignoradas como entrada.

Además, se pueden destacar algunas propiedades adicionales:

* Se fija un ritmo de aprendizaje de 0.01.
* Se elige la función de suma como método de agregación.
* Se utiliza el optimizador `Adam`.
* Se opta por *cross entropy* como función de pérdida.
* Se configura la métrica de precisión para su seguimiento.

In [10]:
from spektral.models.gcn import GCN # Import model
from tensorflow.keras.optimizers import Adam # Import optimizers
from tensorflow.keras.losses import CategoricalCrossentropy # Import loss function from Keras

# Set initial configuration
learning_rate = 0.01
reduction_function = 'sum'
optimizer = Adam(learning_rate=learning_rate)
loss_function = CategoricalCrossentropy(reduction=reduction_function)
metrics_to_evaluate = ['acc']

# Create model from Spektral
model_gcn = GCN(n_labels=dataset.n_labels, n_input_channels=dataset.n_node_features)

# Compile loaded model
model_gcn.compile(
    optimizer=optimizer, loss=loss_function, weighted_metrics=metrics_to_evaluate)

2021-12-11 12:34:29.182540: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Con el modelo listo, es posible proceder a llevar a cabo su entrenamiento. Con el fin de detener el proceso tan pronto como se detecte que la precisión no progresa, se hace uso del callback `EartlyStopping` de Keras con una paciencia de `5`, lo que establece que el proceso se detendrá si pasado dicho número de épocas (*epochs*) no se detecta una mejora en los resultados. De igual manera, se establece un máximo de 50 épocas de entrenamiento.

Para terminar, se añade un callback para enviar los resultados a TensorBoard.

In [11]:
# Eartly stopping settings
early_stopping_patience = 5
max_number_of_epochs = 50

# Fit the model
from tensorflow.keras.callbacks import EarlyStopping
model_gcn.fit(
    loader_training.load(),
    steps_per_epoch=loader_training.steps_per_epoch,
    validation_data=loader_validation.load(),
    validation_steps=loader_validation.steps_per_epoch,
    epochs=max_number_of_epochs,
    callbacks=[
        EarlyStopping(patience=early_stopping_patience,  restore_best_weights=True),
        TensorBoard(get_run_logdir('gcn_spectral'))
    ]
)

Epoch 1/50
1/1 [==============================] - 2s 2s/step - loss: 7.5823 - acc: 0.1357 - val_loss: 2.4473 - val_acc: 0.2160
Epoch 2/50
1/1 [==============================] - 0s 36ms/step - loss: 3.1193 - acc: 0.2357 - val_loss: 1.9142 - val_acc: 0.3280
Epoch 3/50
1/1 [==============================] - 0s 36ms/step - loss: 1.9732 - acc: 0.3429 - val_loss: 1.6517 - val_acc: 0.4180
Epoch 4/50
1/1 [==============================] - 0s 41ms/step - loss: 1.7282 - acc: 0.4500 - val_loss: 1.6359 - val_acc: 0.3840
Epoch 5/50
1/1 [==============================] - 0s 44ms/step - loss: 1.6486 - acc: 0.4643 - val_loss: 1.6203 - val_acc: 0.3920
Epoch 6/50
1/1 [==============================] - 0s 36ms/step - loss: 1.4945 - acc: 0.5000 - val_loss: 1.6186 - val_acc: 0.3780
Epoch 7/50
1/1 [==============================] - 0s 46ms/step - loss: 1.4453 - acc: 0.5571 - val_loss: 1.6002 - val_acc: 0.3920
Epoch 8/50
1/1 [==============================] - 0s 57ms/step - loss: 1.3884 - acc: 0.4643 - val_l

In [12]:
# TODO Añadir número de referencia desde memoria cuando esté todo listo
# TODO Aclarar uso de Adam
# TODO Probar con otros modelos mediante GridSearch (si nos da tiempo)

# TODO Probar sobre conjunto de test

#### Experimento 2 2: ConvGNN espacial mediante paso de mensajes: MPNN

En el ejemplo anterior se ha basado el aprendizaje y la clasificación en la estructura espectral de los datos cargados, es decir, en la estructura de la matriz de adyacencia. Sin embargo, tal y como se explica en la memoria de proyecto, esta aproximación puede ser pobre para escenarios donde las relaciones (aristas) entre entidades (vértices) del grafo tienen un significado o importancia diferente, o cuando el contexto de un nodo, construido mediante los atributos de sus vecinos, es importante para la clasificación.

Para poder preparar este ejemplo, se hará uso de la clase `MessagePassing` de Spektral, que ofrece una API ya preparada para configurar la función de activación y personalizar el comportamiento para distintos juegos de datos. Así, será preciso personalizar:

* Función para la construcción del mensaje pasado entre dos vértices vía la arista que los une. Es conocida como `message` dentro de la API de Spektral.
* Selección de la función de agregación de los mensajes pasados desde cada arista a cada nodo: suma, media, etc. Aparece definida como `aggregate`.

Puesto que las GNN basadas en paso de mensajes requieren sucesivas iteraciones que permitan propagar los mensajes a niveles cada vez más lejanos, la función `propagate` lo ejecuta y computa los atributos de cada nodo tras pasar los mensajes de todas las aristas del grafo y computar la correspondiente función de agregación.

Puesto que Spektral está construido sobre Keras, es preciso, además, implementar algunos de los métodos heredados de la clase `Layer` para definir la capa gráfica espacial:

* `__init__`, donde se define la función de activación, se pasan el resto de hiperparámetros estándar y se guarda el dato del tamaño de la salida, que será usado más adelante.
* `build`, cuya misión es la de inicializar los pesos de la capa mediante la llamada a `add_weight` y asignar el tamaño que tendrá la matriz utilizada para almacenar los pesos dentro de la capa. La matriz de pesos tendrá tantas filas como la entrada y columnas como la salida.
* `call` es el método encargado de ejecutar los cálculos de la capa. Puesto que estamos hablando de paso de mensajes, la salida de la capa será función de las característica de los nodos de entrada y los pesos de la capa. Al final de la misma, en lugar de llamar a la función de activación, se llama a `propagate`, un método disponible dentro de `MessagePassing` encargado de realizar la propagación de los mensajes de cada nodo a sus vecinos.

A estos métodos estándar, se incorporan otros de la propia API de Spektral, especializados en el trabajo con redes gráficas:

* `message`, encargado de componer el mensaje que será pasado entre los nodos. En este ejemplo se hace uso de la información de los nodos vecinos, accesibles mediante `get_j`.
* `aggregate`, con el cometido de definir la función de agregación para los mensajes del vecindario. Aquí se hace uso de la media de todos los mensajes del vecindario, aunque una posible mejora del modelo podría ser el uso de hiperparámetros para encontrar la que mejores reusltados pueda dar. Esta opción es posible desde el propio constructor.
* `update` donde, ahora sí, se hace uso de la función de activaciónn apra actualizar la matriz de pesos.

Con todo, se empieza creando la clase MPNNLayer como herencia `MessagePassing` para preparar la personalización de los citados métodos y definir la capa de paso de mensajes del modelo a usar.

In [13]:
from spektral.layers import MessagePassing

class MPNNLayer(MessagePassing):
    def __init__(self, n_out, activation, **kwargs):
        # Initialize message passing layer with the activation function chosen when creating the model
        super().__init__(activation=activation, **kwargs)
        self.n_out = n_out

    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            shape=(batch_input_shape[0][-1], self.n_out)
        )

    def call(self, inputs):
        x, a = inputs

        # Update node features based on inputs by multiplying node attributes by 
        # the weights stored during build.
        # Kipf 2016
        x = tf.matmul(x, self.kernel)

        # Return propagation result
        return self.propagate(x=x, a=a)

    def message(self, x):
        return self.get_j(x)
    
    def aggregate(self, messages): 
        # We need to return the result of applying the aggregate function over the messages

        # Try to use the mean as aggregate function. We use a scatter mean method for this experiment:
        return spektral.layers.ops.scatter_mean(messages, self.index_i, self.n_nodes)

    def update(self, embeddings):
        return self.activation(embeddings)


Creada la capa, podemos hacer uso de la misma creando un modelo mediante la API funcional de Keras. Para ello, se empieza por configurar algunos parámetros básicos para su funcionamiento:

* Ratio de regularización (o *regularization rate*): con el fin de regularizar los contenidos. Se inicia con un valor de `5e-6`.
* Ratio de aprendizaje (o *learning rate*) con un valor de 0.2.
* Número de epochs de entrenamiento, que son fijadas en 20.
* Nivel de paciencia para el early stopping que será usado más abajo en la etapa de entrenamiento.

Hecho esto, se extran las características principales de los datos: número de nodos, número de careacterísticas por nodo y número de clases para clasificarlos. Estos datos serán usados para instanciar la capa de paso de mensajes.

In [14]:
# Initial setup for the model
l2_regularization_rate = 5e-6
learning_rate = 0.2
epochs = 20
patience = 10

# Input parameters
number_of_nodes = dataset.n_nodes
number_of_node_features = dataset.n_node_features
number_of_labels = dataset.n_labels

Ahora es el momento de definir las entradas. Para ello, se crearán dos tensores:

* `adjacency_matrix` con la matriz de adyacencia, que contendrá la matriz de adyacencia y que permitirá saber las conexiones de cada nodo.
* `x_input` con las características de cada nodo.

Ambas son usadas para definir la capa de salida del modelo, que será construida mediante la clase definida para el MPNNLayer.

In [15]:

# Define input tensors
# We define two input tensors: one for the nodes and one for the adjacency matrix
from keras.layers import Input
adjacency_input = Input(
    (number_of_nodes,), sparse=True, dtype=dataset[0].a.dtype,
    name='adjacency_matrix_input'
)
x_input = Input(
    shape=(number_of_node_features,),
    name='nodes_input'
)

In [16]:
# Define output layer based on the MPNN layer defined previously as MPNNLayer
mpnn_output_layer = MPNNLayer(
    number_of_labels, activation=keras.activations.softmax,
    kernel_regularizer=keras.regularizers.l2(l2_regularization_rate),
    use_bias=False, name='mpnn_layer'
)([x_input, adjacency_input])

In [17]:
# Build and compile model
from keras.models import Model
model_mpnn = Model(
    name='mpnn_gcn',
    inputs=[x_input, adjacency_input],
    outputs=mpnn_output_layer
)
model_mpnn.compile(
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate), 
    loss=keras.losses.CategoricalCrossentropy(),
    weighted_metrics=['acc']
)
model_mpnn.summary()

Model: "mpnn_gcn"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 nodes_input (InputLayer)       [(None, 1433)]       0           []                               
                                                                                                  
 adjacency_matrix_input (InputL  [(None, 2708)]      0           []                               
 ayer)                                                                                            
                                                                                                  
 mpnn_layer (MPNNLayer)         (None, 7)            10031       ['nodes_input[0][0]',            
                                                                  'adjacency_matrix_input[0][0]'] 
                                                                                           

In [18]:
# Train the model
loader_tr = SingleLoader(dataset, sample_weights=dataset.mask_tr)
loader_va = SingleLoader(dataset, sample_weights=dataset.mask_va)
model_mpnn.fit(
    loader_tr.load(),
    steps_per_epoch=loader_tr.steps_per_epoch,
    validation_data=loader_va.load(),
    validation_steps=loader_va.steps_per_epoch,
    epochs=epochs,
    callbacks=[
        EarlyStopping(patience=patience, restore_best_weights=True),
        TensorBoard(get_run_logdir('gcn_spatial'))
    ],
)

Epoch 1/20
1/1 [==============================] - 1s 654ms/step - loss: 0.1002 - acc: 0.1643 - val_loss: 0.2232 - val_acc: 0.6240
Epoch 2/20
1/1 [==============================] - 0s 29ms/step - loss: 0.0153 - acc: 0.9857 - val_loss: 0.1666 - val_acc: 0.7000
Epoch 3/20
1/1 [==============================] - 0s 29ms/step - loss: 0.0032 - acc: 0.9929 - val_loss: 0.1634 - val_acc: 0.7140
Epoch 4/20
1/1 [==============================] - 0s 30ms/step - loss: 0.0011 - acc: 1.0000 - val_loss: 0.1729 - val_acc: 0.7100
Epoch 5/20
1/1 [==============================] - 0s 35ms/step - loss: 4.7404e-04 - acc: 1.0000 - val_loss: 0.1853 - val_acc: 0.7060
Epoch 6/20
1/1 [==============================] - 0s 32ms/step - loss: 2.4123e-04 - acc: 1.0000 - val_loss: 0.1980 - val_acc: 0.7100
Epoch 7/20
1/1 [==============================] - 0s 36ms/step - loss: 1.3434e-04 - acc: 1.0000 - val_loss: 0.2102 - val_acc: 0.7100
Epoch 8/20
1/1 [==============================] - 0s 34ms/step - loss: 7.9853e-05 - 

Una vez entrenado el modelo, se puede evaluar contra el 

In [21]:
loader_te = SingleLoader(dataset, sample_weights=dataset.mask_te)
results = model_mpnn.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)

print(f'Test loss: {results[0]}')
print(f'Test accuracy: {results[1]}')

1/1 [==============================] - 0s 15ms/step - loss: 0.3277 - acc: 0.7330
Test loss: 0.32770317792892456
Test accuracy: 0.7329999804496765
